<a href="https://colab.research.google.com/github/surajsarkar/deepLearning/blob/main/notebooks/04_exercise_deep_learning_with_tensor_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transfer learning with full data**

## **Download the data**

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

--2022-06-14 09:40:03--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 142.250.99.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip’

10_food_classes_all 100%[===================>] 495.13M   317MB/s    in 1.6s    

2022-06-14 09:40:05 (317 MB/s) - ‘10_food_classes_all_data.zip’ saved [519183241/519183241]



In [ ]:
# Unzip the file
import zipfile

with zipfile.ZipFile("10_food_classes_all_data.zip") as zip_ref:
  zip_ref.extractall()

In [ ]:
# Import modules
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam 


In [ ]:
# Create Image generators
train_dir = "10_food_classes_all_data/train/"
test_dir = "10_food_classes_all_data/test/"


train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Traning Data \n")
train_data = train_datagen.flow_from_directory(directory=train_dir,
                           target_size = (224, 224)
                           )

print("Test Data \n")
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                             target_size=(224, 224))

Traning Data 

Found 7500 images belonging to 10 classes.
Test Data 

Found 2500 images belonging to 10 classes.


In [ ]:
import tensorflow_hub as hub 

In [ ]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
from tensorflow.keras import layers

In [ ]:
# Create function to make and compile model.

# def create_model(hub_url):
#   feature_extractor_layer = hub.KerasLayer(hub_url, 
#                                            trainable=False, 
#                                            name="feature_extractor_layer", 
#                                            input_shape=(224, 224, 3))
#   model = Sequential([
#     feature_extractor_layer,
#     layers.Dense(10, activation="softmax", name="Output layer")  
#   ])

#   return model


In [ ]:
IMAGE_SIZE = (224, 224)

In [ ]:
def create_model(model_url, num_classes=10):

  # Download the pretrained model and save it as a Keras layer 
  feature_extractor_layer = hub.KerasLayer(model_url, 
                                           trainable=False, # freeze the already trained pattern
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SIZE + (3,))
  
  # Create our own model 
  model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(num_classes, activation="softmax", name="output_layer")
  ])
  
  return model

In [ ]:
train_data.num_classes

10

In [ ]:
resnet_model = create_model(model_url=resnet_url, num_classes=train_data.num_classes)

resnet_model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)
resnet_model.fit(train_data,
                 epochs=5,
                 steps_per_epoch=len(train_data),
                 validation_data=test_data,
                 validation_steps=len(test_data))

Epoch 1/5
235/235 [==============================] - 70s 237ms/step - loss: 0.9051 - accuracy: 0.7052 - val_loss: 0.5233 - val_accuracy: 0.8280
Epoch 2/5
235/235 [==============================] - 53s 226ms/step - loss: 0.5353 - accuracy: 0.8252 - val_loss: 0.4693 - val_accuracy: 0.8424
Epoch 3/5
235/235 [==============================] - 51s 217ms/step - loss: 0.4359 - accuracy: 0.8581 - val_loss: 0.4700 - val_accuracy: 0.8416
Epoch 4/5
235/235 [==============================] - 48s 205ms/step - loss: 0.3709 - accuracy: 0.8841 - val_loss: 0.4611 - val_accuracy: 0.8456
Epoch 5/5
235/235 [==============================] - 48s 204ms/step - loss: 0.3175 - accuracy: 0.9048 - val_loss: 0.4615 - val_accuracy: 0.8484


In [ ]:
efficientnet_model = create_model(efficientnet_url)

efficientnet_model.compile(
    loss="categorical_crossentropy",
    optimizer = Adam(),
    metrics = ["accuracy"]
)


efficientnet_model.fit(train_data,
                       epochs=5,
                       steps_per_epoch=len(train_data),
                       validation_data=test_data,
                       validation_steps=len(test_data))

Epoch 1/5
235/235 [==============================] - 56s 204ms/step - loss: 0.8120 - accuracy: 0.7780 - val_loss: 0.4028 - val_accuracy: 0.8860
Epoch 2/5
235/235 [==============================] - 47s 200ms/step - loss: 0.4473 - accuracy: 0.8653 - val_loss: 0.3292 - val_accuracy: 0.9032
Epoch 3/5
235/235 [==============================] - 46s 196ms/step - loss: 0.3744 - accuracy: 0.8888 - val_loss: 0.3031 - val_accuracy: 0.9068
Epoch 4/5
235/235 [==============================] - 46s 197ms/step - loss: 0.3294 - accuracy: 0.9017 - val_loss: 0.2866 - val_accuracy: 0.9124
Epoch 5/5
235/235 [==============================] - 46s 196ms/step - loss: 0.2984 - accuracy: 0.9099 - val_loss: 0.2761 - val_accuracy: 0.9148


In [ ]:
mobilenet_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5"

In [ ]:
mobilenet_model = create_model(mobilenet_url, train_data.num_classes)

mobilenet_model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]  
)

mobilenet_model.fit(train_data, 
                     epochs=5, 
                     steps_per_epoch=len(train_data),
                     validation_data=test_data,
                     validation_steps=len(test_data))

Epoch 1/5
235/235 [==============================] - 49s 196ms/step - loss: 0.8717 - accuracy: 0.7228 - val_loss: 0.5129 - val_accuracy: 0.8320
Epoch 2/5
235/235 [==============================] - 45s 193ms/step - loss: 0.5378 - accuracy: 0.8288 - val_loss: 0.4608 - val_accuracy: 0.8508
Epoch 3/5
235/235 [==============================] - 46s 195ms/step - loss: 0.4580 - accuracy: 0.8528 - val_loss: 0.4322 - val_accuracy: 0.8584
Epoch 4/5
235/235 [==============================] - 45s 191ms/step - loss: 0.4017 - accuracy: 0.8705 - val_loss: 0.4149 - val_accuracy: 0.8676
Epoch 5/5
235/235 [==============================] - 45s 193ms/step - loss: 0.3623 - accuracy: 0.8847 - val_loss: 0.4153 - val_accuracy: 0.8648
